***
@author: jmeier<br>
date: 10. january 2024<br>
short description: <br>
-  Displays traveling time and distance to UKR using travel by car or transit
-  Can be done via Google API or data from csv file
***

<div class="alert alert-box alert-danger"><b>Caveat:</b> Google distance matrix service is fee-based!<br>
<b>Google API key must be entered in file auth.py</b></div>

In [40]:
import sqlite3
import pandas as pd
import geopandas as gpd
pd.options.mode.chained_assignment = None  # default='warn'  --> catches unnecessary pandas warning
import googlemaps   
from datetime import datetime
from auth import auth

In [41]:
## Loads personalGgoogle API key and initializes googlemaps Client with it
auth_google = auth()
gmaps = googlemaps.Client(key=auth_google)

In [42]:
# Sets the destination to UKR and the depature time
destination = ["Universitätsklinikum Regensburg Franz-Josef-Strauß-Allee 11, 93053 Regensburg, Germany"]

departure_time = datetime(2024,1,11,7,0,0,0)

In [43]:
# Reads in .shp (shape) file with the shapes of all "Landkreise" of germany. Selects bavarian "Landkreise"
states = gpd.read_file("../shapefile/DEU_adm3.shp")
bavaria_lkrs = states.loc[states["NAME_1"] == "Bayern"]

# Adds empty columns for distance and time for travel by car or transit
bavaria_lkrs["dist_car"] = 0
bavaria_lkrs["time_car"] = 0
bavaria_lkrs["dist_trans"] = 0
bavaria_lkrs["time_trans"] = 0
bavaria_lkrs.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 96 entries, 44 to 139
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID_0        96 non-null     int64   
 1   ISO         96 non-null     object  
 2   NAME_0      96 non-null     object  
 3   ID_1        96 non-null     int64   
 4   NAME_1      96 non-null     object  
 5   ID_2        96 non-null     int64   
 6   NAME_2      96 non-null     object  
 7   ID_3        96 non-null     int64   
 8   NAME_3      96 non-null     object  
 9   TYPE_3      96 non-null     object  
 10  ENGTYPE_3   96 non-null     object  
 11  NL_NAME_3   0 non-null      object  
 12  VARNAME_3   12 non-null     object  
 13  geometry    96 non-null     geometry
 14  dist_car    96 non-null     int64   
 15  time_car    96 non-null     int64   
 16  dist_trans  96 non-null     int64   
 17  time_trans  96 non-null     int64   
dtypes: geometry(1), int64(8), object(9)
memory usag

In [44]:
#bavaria_lkrs.to_file("test.shp")  

In [45]:
# If no valid travel is returned by Google (ZERO_RESULTS) the value "None" is returned, else the travel distance or time value
# results: Googlemaps distance matrix result; community: String Name of community tested; 
#nr: Position of community in "origins"-List. Here always = 0. dist_or_time: If "distance" or time (="duration") should be returned
def distance_time(results, community, nr, dist_or_time):
    if results["rows"][0]["elements"][nr]["status"] == "ZERO_RESULTS":
        print(community, " without results for #", nr)
        return None
    elif results["rows"][0]["elements"][nr]["status"] == "OK":
        return results["rows"][0]["elements"][nr][dist_or_time]["value"]
    else:
        print("Unknown status:")
        print(results["rows"][0]["elements"][nr]["status"])

In [49]:
# Here can be selected if googlemaps should be used to get the travel distances or if data should be read from exisiting shp file
do_gmaps_request = False

# Returns GeoDataFrame with the travel distances and times for travel by car or transit for all "Landkreise" to the UKR
if do_gmaps_request:
    for lkr in range(len(bavaria_lkrs)):
        if bavaria_lkrs.iloc[lkr]["TYPE_3"] == "Kreisfreie Städte":
            type_lkr = "Stadt"
            name_lkr = bavaria_lkrs.iloc[lkr]["NAME_3"].split(" ")[0]
            
        elif bavaria_lkrs.iloc[lkr]["TYPE_3"] == "Landkreise":
            type_lkr = "Landkreis"
            name_lkr = bavaria_lkrs.iloc[lkr]["NAME_3"]
        else:            
            print(bavaria_lkrs.iloc[lkr]["TYPE_3"])

        origin = type_lkr + " " + name_lkr
        #print(origin)

        results_car = gmaps.distance_matrix(origins=origin, destinations=destination, departure_time=departure_time, mode="driving")
        results_transit = gmaps.distance_matrix(origins=origin, destinations=destination, departure_time=departure_time, mode="transit")

        #Travel times will be calculated by google api
        #print("b:", bavaria_lkrs.iloc[lkr]["travel_distance_car"])
        bavaria_lkrs.iat[lkr, 14] = distance_time(results_car, origin, 0, "distance") #car travel distance
        bavaria_lkrs.iat[lkr, 15] = distance_time(results_car, origin, 0, "duration")    #car travel time

        bavaria_lkrs.iat[lkr, 16] = distance_time(results_transit, origin, 0, "distance") #transit travel distance
        bavaria_lkrs.iat[lkr, 17] = distance_time(results_transit, origin, 0, "duration") #transit travel time
        #print("a:",bavaria_lkrs.iloc[lkr]["travel_distance_car"])
        #break
        if (lkr+1) % 10 ==0:
            print(round((lkr+1)/len(bavaria_lkrs)*100, 1),"%")
    bavaria_lkrs.to_file("travel_time_lkr.shp")  
    bavaria_lkrs.head()

else:
    print("read shp")
    
    bavaria_lkrs = gpd.read_file("travel_time_lkr.shp")
    


read shp


In [105]:
# For some "Landkreise" no travel by transit could be calculated. here manually update (travel times from cities within the Landkreis were used)
freyung_grafenau_id = bavaria_lkrs.loc[bavaria_lkrs["NAME_3"] == "Freyung-Grafenau"].index[0]
bavaria_lkrs.at[freyung_grafenau_id, "dist_trans"] = 127844 # From Grafenau
bavaria_lkrs.at[freyung_grafenau_id, "time_trans"] = 9028
main_spessart_id = bavaria_lkrs.loc[bavaria_lkrs["NAME_3"] == "Main-Spessart"].index[0]
bavaria_lkrs.at[main_spessart_id, "dist_trans"] = 231942 # From Karlstadt
bavaria_lkrs.at[main_spessart_id, "time_trans"] = 13036
rosenheim_id = bavaria_lkrs.loc[bavaria_lkrs["NAME_3"] == "Rosenheim"].index[0]
bavaria_lkrs.at[rosenheim_id, "dist_trans"] = 203211 # From Rosenheim Stadt
bavaria_lkrs.at[rosenheim_id, "time_trans"] = 9974
aschaffensburg_id = bavaria_lkrs.loc[bavaria_lkrs["NAME_3"] == "Aschaffenburg"].index[0]
bavaria_lkrs.at[aschaffensburg_id, "dist_trans"] = 286497 # From Aschaffenburg
bavaria_lkrs.at[aschaffensburg_id, "time_trans"] = 12825
#print(main_spessart_id)
bavaria_lkrs.loc[pd.isna(bavaria_lkrs["time_trans"])]
#bavaria_lkrs.loc[bavaria_lkrs["NAME_3"] == "Rosenheim Städte"]

Empty GeoDataFrame
Columns: [ID_0, ISO, NAME_0, ID_1, NAME_1, ID_2, NAME_2, ID_3, NAME_3, TYPE_3, ENGTYPE_3, NL_NAME_3, VARNAME_3, dist_car, time_car, dist_trans, time_trans, geometry, time_car_h, time_trans_h, dist_car_km, dist_trans_km]
Index: []

[0 rows x 22 columns]

In [106]:
# Calculates the time in hours instead of seconds and the distance in km instead of meters (for tooltip in graphs)
bavaria_lkrs["time_car_h"] = round(bavaria_lkrs["time_car"] / 3600, 1)
bavaria_lkrs["time_trans_h"] = round(bavaria_lkrs["time_trans"] / 3600, 1)
bavaria_lkrs["dist_car_km"] = round(bavaria_lkrs["dist_car"] / 1000, 1)
bavaria_lkrs["dist_trans_km"] = round(bavaria_lkrs["dist_trans"] / 3600, 1)
bavaria_lkrs.head()

ID_0  ISO   NAME_0  ID_1  NAME_1  ID_2         NAME_2  ID_3  \
0     86  DEU  Germany     2  Bayern     5  Mittelfranken    45   
1     86  DEU  Germany     2  Bayern     5  Mittelfranken    46   
2     86  DEU  Germany     2  Bayern     5  Mittelfranken    47   
3     86  DEU  Germany     2  Bayern     5  Mittelfranken    48   
4     86  DEU  Germany     2  Bayern     5  Mittelfranken    49   
5     86  DEU  Germany     2  Bayern     5  Mittelfranken    50   
6     86  DEU  Germany     2  Bayern     5  Mittelfranken    51   
7     86  DEU  Germany     2  Bayern     5  Mittelfranken    52   
8     86  DEU  Germany     2  Bayern     5  Mittelfranken    53   
9     86  DEU  Germany     2  Bayern     5  Mittelfranken    54   
10    86  DEU  Germany     2  Bayern     5  Mittelfranken    55   
11    86  DEU  Germany     2  Bayern     5  Mittelfranken    56   
12    86  DEU  Germany     2  Bayern     6   Niederbayern    57   
13    86  DEU  Germany     2  Bayern     6   Niederbayern    58   
14    86  DEU  Germany     2  Bayern     6   Niederbayern    59   

                     NAME_3             TYPE_3  ...  \
0            Ansbach Städte  Kreisfreie Städte  ...   
1                   Ansbach         Landkreise  ...   
2        Erlangen-Höchstadt         Landkreise  ...   
3           Erlangen Städte  Kreisfreie Städte  ...   
4              Fürth Städte  Kreisfreie Städte  ...   
5                     Fürth         Landkreise  ...   
6           Nürnberger Land         Landkreise  ...   
7    Neustadt-Bad Windsheim         Landkreise  ...   
8          Nuremberg Städte  Kreisfreie Städte  ...   
9                      Roth         Landkreise  ...   
10         Schwabach Städte  Kreisfreie Städte  ...   
11  Weißenburg-Gunzenhausen         Landkreise  ...   
12               Deggendorf         Landkreise  ...   
13        Dingolfing-Landau         Landkreise  ...   
14         Freyung-Grafenau         Landkreise  ...   

                            VARNAME_3 dist_car time_car  dist_trans  \
0                                None   149700     6001    184443.0   
1                                None   153319     5958    160474.0   
2                                None   140226     5726    186085.0   
3                                None   120861     4705    128612.0   
4                                None   116277     4979    112816.0   
5                                None   115532     4752    152937.0   
6                                None   103479     4063    158176.0   
7   Neustadt a.d.Aisch-Bad Windsheim|   159433     6790    161341.0   
8                            Nürnberg   111439     4706    105423.0   
9                                None   119317     4551    167113.0   
10                               None   109162     4333    120285.0   
11                               None   144937     5612    194386.0   
12                               None    72197     2767     76881.0   
13                               None    76648     3391     84186.0   
14                               None   139863     5686    127844.0   

    time_trans                                           geometry  time_car_h  \
0       8474.0  POLYGON ((10.59538 49.33940, 10.60123 49.33205...         1.7   
1      12562.0  POLYGON ((10.84725 49.19458, 10.84160 49.19461...         1.7   
2      10186.0  POLYGON ((10.82027 49.53257, 10.82022 49.53629...         1.6   
3       9194.0  POLYGON ((10.94087 49.53362, 10.94085 49.53735...         1.3   
4       7994.0  POLYGON ((10.99869 49.43301, 10.99296 49.44049...         1.4   
5       7748.0  POLYGON ((10.99869 49.43301, 11.00443 49.42555...         1.3   
6       7613.0  POLYGON ((11.18192 49.47550, 11.17624 49.47925...         1.1   
7       9794.0  POLYGON ((10.08806 49.53819, 10.09362 49.54193...         1.9   
8       5114.0  POLYGON ((11.00461 49.37310, 10.99891 49.37683...         1.3   
9       7673.0  POLYGON ((11.25386 49.31978, 11.24821 49.31980...         1.3   
10      8414.0  POLYGO

In [113]:
# Travel by car map. Generates a heatmap based on the column "time_car_h". Saves it as html file
lkrs_overview_map = bavaria_lkrs.explore(
                    column="time_car_h",
                    cmap="OrRd", 
                    tooltip=["NAME_3", "time_car_h", "dist_car_km"],
                    vmin=0,
                    vmax=3
                    )
lkrs_overview_map.save("car_to_rgb_lkr.html")
lkrs_overview_map

In [120]:
# Travel by transit map. Generates a heatmap based on the column "time_trans_h". Saves it as html file
lkrs_overview_map_transit = bavaria_lkrs.explore(
                    column="time_trans_h",
                    cmap="OrRd", 
                    tooltip=["NAME_3", "time_trans_h", "dist_trans_km"],
                    vmin=0,
                    vmax=3,
                    #tiles="MapQuest Open Aerial"
                    )
lkrs_overview_map_transit.save("transit_to_rgb_lkr.html")
lkrs_overview_map_transit